# BigQuery

In [1]:
from google.cloud import bigquery
import os

### Setting up the Google credentials here:

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bqexercise-32c5a833fe37.json"

In [3]:
client = bigquery.Client()

### Calling the API

In [4]:
# Constructing a reference to the 'hacker news' dataset

df_ref = client.dataset('hacker_news', project = 'bigquery-public-data')

In [5]:
# Building API request

df_hn = client.get_dataset(df_ref)

In [10]:
# List tables in 'hacker news' dataset

tables = list(client.list_tables(df_hn))

# Print names of all tables in dataset:
for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


In [12]:
# Building reference to the full table: 
table_ref = df_ref.table('full')

# API request: fetch the table.
table = client.get_table(table_ref)

### Taking a look at the table schema

In [13]:
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', (), ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', (), ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', (), ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', (), ()),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", (), ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', (), ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', (), ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', (), ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', (), ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", (), ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', (), ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', (), ()),
 SchemaField('ranking', 'INTEGER', '

In [14]:
# Previewing the first five lines of the "FULL" table:

client.list_rows(table, max_results = 5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,I read at some point that a lot of interesting...,None,JackC,NaN,1337196418,2012-05-16 19:26:58+00:00,comment,3983591,3983475,NaN,NaN,None
1,None,None,"If you remove ""fraudulent"" from the definition...",None,natrius,NaN,1319409058,2011-10-23 22:30:58+00:00,comment,3147488,3147469,NaN,NaN,None
2,None,None,Using Gentoo stable in production right now. I...,None,anonbanker,NaN,1451111517,2015-12-26 06:31:57+00:00,comment,10793491,10784084,NaN,NaN,None
3,None,None,Considering that -272 C is very close to absol...,None,dllu,NaN,1451111489,2015-12-26 06:31:29+00:00,comment,10793490,10793431,NaN,NaN,None
4,None,None,Also the whole Gemset thing is basically redun...,None,lucaspiller,NaN,1451112020,2015-12-26 06:40:20+00:00,comment,10793497,10793246,NaN,NaN,None


In [18]:
# If you want to select specific columns: 

client.list_rows(table, selected_fields=table.schema[2:3], max_results = 5).to_dataframe()

,text
0,I read at some point that a lot of interesting...
1,"If you remove ""fraudulent"" from the definition..."
2,Using Gentoo stable in production right now. I...
3,Considering that -272 C is very close to absol...
4,Also the whole Gemset thing is basically redun...


***

# Exploring a new dataset.

In [20]:
# Checking table name

dataset_ref = client.dataset('openaq', project = 'bigquery-public-data')
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

global_air_quality


In [22]:
# Fetching table and previewing it:

table_ref = dataset_ref.table('global_air_quality')
table = client.get_table(table_ref)
client.list_rows(table, max_results = 5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


#### Constructing the query: 

In [26]:
# Query to select ALL the items from the "city" column where "country" is "US":

query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

Use the ` instead of '' when """"""

### Submitting the query to the dataset

In [27]:
client = bigquery.Client()

# Set up the query:

query_job = client.query(query)

#Convert results in to a Pandas DF:
us_cities = query_job.to_dataframe()

In [29]:
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     88
Houston                                     82
Los Angeles-Long Beach-Santa Ana            68
Riverside-San Bernardino-Ontario            60
New York-Northern New Jersey-Long Island    60
Name: city, dtype: int64

### Here's how to do more queries

Select with a comma between the names!

In [30]:
query = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

You can also select all using *

In [31]:
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

To see how much data you're going to scan... use *QueryJobConfig* object and set *dry_run* to True.

In [33]:
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

# Create a QJC object to estimate size of query without running it:
dry_run_config = bigquery.QueryJobConfig(dry_run = True)

# API request
dry_run_query_job = client.query(query, job_config = dry_run_config)
print('This query will process {} bytes.'.format(dry_run_query_job.total_bytes_processed))

This query will process 2022805 bytes.


#### You can also limit how much data you are willing to scan!

In [34]:
# Run if query is less than 1MB:

ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = ONE_MB)

# Set up query (It will only run if < 1MB):
safe_query_job = client.query(query, job_config = safe_config)

# API Request
safe_query_job.to_dataframe()

InternalServerError: 500 Query exceeded limit for bytes billed: 1000000. 10485760 or higher required.

(job ID: 2ad6d038-76cc-4e94-addd-579c1892da57)

                 -----Query Job SQL Follows-----                  

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:        SELECT *
   3:        FROM `bigquery-public-data.openaq.global_air_quality`
   4:        WHERE country = 'US'
   5:        
    |    .    |    .    |    .    |    .    |    .    |    .    |

***

### Group By, Having, Count

`COUNT()` - returns a count of things! This is an **aggregate** function.

`GROUP BY()` - takes the name of one or more columns and treats all rows with thesame value in the column as a single group when you apply an aggregate function like COUNT(). 

*NOTE: It doesn't make sense to use `GROUP BY` without an aggregate function!*

`GROUP BY` ... HAVING - This is used in combination with GROUP BY() to ignore groups that don't meet certain parameters.

In [43]:
# Re-using the Hacker News Data:

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "comments" table
client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


In [44]:
# Query to select comments that received more than 10 replies
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """

In [45]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
popular_comments.head()

,parent,f0_
0,166905,55
1,5100679,41
2,5165016,109
3,4098430,42
4,7970366,455


### Aliasing and other improvements:

Change the column names to something that you can actually understand using `AS`. 

If you're unsure of what to put in `COUNT()`, just put `COUNT(1)`.

In [47]:
improved_query = """
        SELECT parent, COUNT(1) as NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
        HAVING COUNT(1) > 10
        """

In [49]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(improved_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_comments.head()

,parent,NumPosts
0,6683866,39
1,6627329,46
2,3476843,49
3,7234010,48
4,2932956,76


***

### Order By

`Order By` is usually the *last* clause in your query.

Use the **DESC** argument to reverse the A-Z default (ASC) order.

In BQ, dates are stored as: 

**DATE** 
> YYYY-MM-DD

or **DATETIME**. 
> YYYY-MM-DD:H:M:S:MS

#### EXTRACT

You can extract parts of a date (like year or day) using `EXTRACT`

### EXAMPLE DATASET:

In [50]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "nhtsa_traffic_fatalities" dataset
dataset_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "accident_2015" table
table_ref = dataset_ref.table("accident_2015")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "accident_2015" table
client.list_rows(table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,19,Iowa,190204,1,1,0,0,0,1,1,...,2,0,None,0,None,0,None,1,1,2015-09-11 20:20:00+00:00
1,19,Iowa,190233,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-11-01 00:30:00+00:00
2,19,Iowa,190179,1,1,0,0,0,2,2,...,1,0,None,0,None,0,None,1,0,2015-05-04 16:18:00+00:00
3,19,Iowa,190248,1,1,0,0,0,4,4,...,99,0,None,0,None,0,None,2,0,2015-11-17 12:26:00+00:00
4,19,Iowa,190231,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2015-10-31 04:49:00+00:00


#### Which day of the week has the most fatal accidents?

`consecutive_number` contains unique IDs for each accident. 

`timestamp_of_crash` contains the date of the accident in **DATETIME** format.

In [55]:
query = """
        SELECT COUNT(consecutive_number) as num_accidents,
                EXTRACT (DAYOFWEEK FROM timestamp_of_crash) as day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

In [56]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

# Print the DataFrame
accidents_by_day

,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3
